In [ ]:
import matplotlib.pyplot as plt
from random import seed
from random import randrange
from random import random
from csv import reader
from math import exp
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import learning_curve
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler


# MLP construction
# Load data
def load():
    dataset = []
    with open("wdbc.csv", 'r') as file:
        csv_reader = reader(file)
        for i in csv_reader:
            dataset.append(i)
    return dataset

# Parses the string values as floats
def StringToFloat(dataset, column):
    for i in dataset:
        i[column] = float(i[column].strip())

# Parses the string values as integers
def StringToInt(dataset, column):
    class_values = [i[column] for i in dataset]
    unique = set(class_values)
    lookup = {}

    for i, value in enumerate(unique):
        lookup[value] = i
    for j in dataset:
        j[column] = lookup[j[column]]
    return lookup

# Find the min and max values per column
def MinMax(dataset):
    stats = [[min(i), max(i)] for i in zip(*dataset)]
    return stats

# Rescale cols to the range [0-1]
def normalize(dataset, minmax):
    for row in dataset:
        for i in range(len(row) - 1):
            row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])

# Split dataset into k-folds
def xvalidation(dataset, n_folds):
    dataset_split = []
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    for i in range(n_folds):
        fold = []
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split

# Calculate accuracy
def accuracy(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

# Calculate error
def error(actual, predicted):
    incorrect = 0
    for i in range(len(actual)):
        if actual[i] != predicted[i]:
            incorrect += 1
    return incorrect / float(len(actual)) * 100.0

# Eval algorithm using cross validation
def evalalgorithm(dataset, algorithm, n_folds, *args):
    folds = xvalidation(dataset, n_folds)
    scores = []
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        test_set = []
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
        acc = accuracy(actual, predicted)
        scores.append(acc)
    return scores

# Eval algorithm errors w/ cross validation
def negeval(dataset, algorithm, n_folds, *args):
    folds = xvalidation(dataset, n_folds)
    misses = []
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        test_set = []
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
        err = error(actual, predicted)
        misses.append(err)
    return misses

# Calculate neuron activation
def activate(weights, inputs):
    activation = weights[-1]
    for i in range(len(weights) - 1):
        activation += weights[i] * inputs[i]
    return activation

# Transfer neuron activation
def transfer(activation):
    return 1.0 / (1.0 + exp(-activation))

# Forward propagate input to a network output
def forward_propagate(network, row):
    inputs = row
    for layer in network:
        new_inputs = []
        for neuron in layer:
            activation = activate(neuron['weights'], inputs)
            neuron['output'] = transfer(activation)
            new_inputs.append(neuron['output'])
        inputs = new_inputs
    return inputs

# Calculate the derivative of an neuron output
def marginalization(output):
    return output * (1.0 - output)

# Backpropagate error and store it in neurons
def backward_propagate_error(network, expected):
    for i in reversed(range(len(network))):
        layer = network[i]
        errors = []
        if i != len(network) - 1:
            for j in range(len(layer)):
                error = 0.0
                for k in network[i + 1]:
                    error += (k['weights'][j] * k['delta'])
                errors.append(error)
        else:
            for j in range(len(layer)):
                k = layer[j]
                errors.append(expected[j] - k['output'])
        for j in range(len(layer)):
            k = layer[j]
            k['delta'] = errors[j] * marginalization(k['output'])

# Update network weights with error
def update_weights(network, row, l_rate):
    for i in range(len(network)):
        inputs = row[:-1]
        if i != 0:
            inputs = [k['output'] for k in network[i - 1]]
        for neuron in network[i]:
            for j in range(len(inputs)):
                neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
            neuron['weights'][-1] += l_rate * neuron['delta']

# Train network for a fixed # of epochs
def train_network(network, train, l_rate, n_epoch, n_outputs):
    for epoch in range(n_epoch):
        for row in train:
            outputs = forward_propagate(network, row)
            expected = [0 for i in range(n_outputs)]
            expected[row[-1]] = 1
            backward_propagate_error(network, expected)
            update_weights(network, row, l_rate)

# Initialize network
def initialize_network(n_inputs, n_hidden, n_outputs):
    network = []
    hidden_layer = [{'weights': [random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
    network.append(hidden_layer)
    output_layer = [{'weights': [random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
    network.append(output_layer)
    return network

# Make prediction with network
def predict(network, row):
    outputs = forward_propagate(network, row)
    return outputs.index(max(outputs))

# Back Propagation Algorithm w/ Stochastic Gradient Descent
def back_propagation(train, test, l_rate, n_epoch, n_hidden):
    n_inputs = len(train[0]) - 1
    n_outputs = len(set([row[-1] for row in train]))
    network = initialize_network(n_inputs, n_hidden, n_outputs)
    train_network(network, train, l_rate, n_epoch, n_outputs)
    predictions = []
    for i in test:
        prediction = predict(network, i)
        predictions.append(prediction)
    return predictions


In [3]:
#SVM Construction
from sklearn.datasets import load_breast_cancer
cancer = load_breast_cancer()
#Load input features as DataFrame
df_features = pd.DataFrame(cancer['data'], columns = cancer['feature_names'])
#Add output variable 'target' into Data Frame
df_target = pd.DataFrame(cancer['target'], columns = ['Cancer'])
df_features.head()
#split up into test/train
X_train, X_test, y_train, y_test = train_test_split(df_features, np.ravel(df_target), test_size=0.30, random_state=101)
#import SVC model w linear kernel
model = svm.SVC(kernel='linear')
#Train the model using fit method
model.fit(X_train, y_train)
predictions = model.predict(X_test)


#SVM with 5 fold cross validation
X_for_Kfold = np.array(X_train)
y_for_Kfold = np.array(y_train)
kf = KFold(n_splits=5)

for i, j in kf.split(X_for_Kfold):
    X_train2, X_test2 = X_for_Kfold[i], X_for_Kfold[j]
    y_train2, y_test2 = y_for_Kfold[i], y_for_Kfold[j]


stdScaler = StandardScaler()
# pre-process the data
stdScaler.fit(X_train2)
X_train2 = stdScaler.transform(X_train2)
X_test2 = stdScaler.transform(X_test2)
# Train the model using the training sets
model.fit(X_train2, y_train2)
# Predict the response for test dataset
predictions2 = model.predict(X_test2)

# To test SVM

In [5]:
#Test - SVM
print('SVM report: ')
print(classification_report(y_test, predictions))
print("Total SVM model accuracy: ", metrics.accuracy_score(y_test, predictions))
print("Total 5-fold SVM model accuracy: ",
      metrics.accuracy_score(y_test2, predictions2))


SVM report: 
              precision    recall  f1-score   support

           0       0.95      0.91      0.93        66
           1       0.94      0.97      0.96       105

    accuracy                           0.95       171
   macro avg       0.95      0.94      0.94       171
weighted avg       0.95      0.95      0.95       171

Total SVM model accuracy:  0.9473684210526315
Total 5-fold SVM model accuracy:  0.9873417721518988


# To test MLP (and plot) 

In [6]:
# Test - MLP
seed(1)
new_dataset = load()
new = list()
dataset = []

for i in new_dataset:
    new = i[2:]
    new.append(i[1])
    dataset.append(new)

for i in range(len(dataset[0]) - 1):
    StringToFloat(dataset, i)

StringToInt(dataset, len(dataset[0]) - 1)
min_max = MinMax(dataset)
normalize(dataset, min_max)

n_folds = 5
l_rate = 0.1
n_epoch = 100
n_hidden = 5

accs = []
errs = []
datasizes = []

x = .1
print('Train MLP: ')
while x <= 1.0:
    print('epoch: ', x)
    temp = dataset[:int(len(dataset) * x)]
    scores = evalalgorithm(temp, back_propagation, n_folds, l_rate, n_epoch, n_hidden)
    misses = negeval(temp, back_propagation, n_folds, l_rate, n_epoch, n_hidden)
    MA = sum(scores) / float(len(scores))
    ME = sum(misses) / float(len(misses))

    accs.append(MA)
    errs.append(ME)
    datasizes.append(x)
    x += 0.1

    print('Errors = %s' % misses)
    print('Scores = %s' % scores)
    print('Mean Accuracy = %.3f%%' % MA)
    print('Mean Error = %.3f%%' % ME)
    print('-------------------')

plt.plot(datasizes,accs, 'b', label = 'Learning curve for training')
plt.plot(datasizes,errs, 'c', label = 'Learning curve for validation set')
plt.ylabel('Error')
plt.xlabel('Size of Data')
plt.title('Learning Curve for Multilayer Perceptron Model', fontsize = 18, y = 1.03)
plt.style.use('dark_background')
plt.legend()
#plt.show()
print('------------------------------------------------------------------')

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x90 in position 11: invalid start byte